In [2]:
pip install py2neo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 1.5 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from py2neo import Graph, Node, Relationship, NodeMatcher

class KnowledgeGraph:
    def __init__(self, uri="neo4j+s://d5800584.databases.neo4j.io", user="neo4j", password="wPQh0QgH3kzqb8iBSAenCL54ZX7suzKy4vt4_jdj6tU"):
        self.graph = Graph(uri, auth=(user, password))
        self.matcher = NodeMatcher(self.graph)

    # Clear the entire graph (optional)
    def clear_graph(self):
        self.graph.run("MATCH (n) DETACH DELETE n")
        print("Graph cleared.")

    # Batch add multiple users
    def add_users(self, user_names):
        tx = self.graph.begin()
        for name in user_names:
            user = Node("User", name=name)
            tx.merge(user, "User", "name")  # Ensures no duplicates
            print(f"User '{name}' added.")
        tx.commit()

    # Batch add multiple countries
    def add_countries(self, countries):
        tx = self.graph.begin()
        for country in countries:
            country_node = Node("Country", name=country)
            tx.merge(country_node, "Country", "name")
            print(f"Country '{country}' added.")
        tx.commit()

    # Batch add multiple continents
    def add_continents(self, continents):
        tx = self.graph.begin()
        for continent in continents:
            continent_node = Node("Continent", name=continent)
            tx.merge(continent_node, "Continent", "name")
            print(f"Continent '{continent}' added.")
        tx.commit()

    # Batch add multiple languages
    def add_languages(self, languages):
        tx = self.graph.begin()
        for language in languages:
            language_node = Node("Language", name=language)
            tx.merge(language_node, "Language", "name")
            print(f"Language '{language}' added.")
        tx.commit()

    # Add relationships between users and countries they visited
    def add_visited_relationship(self, user_name, country_name):
        user = self.matcher.match("User", name=user_name).first()
        country = self.matcher.match("Country", name=country_name).first()
        if user and country:
            rel = Relationship(user, "VISITED", country)
            self.graph.merge(rel)
            print(f"User '{user_name}' visited the country '{country_name}'.")
        else:
            print(f"User '{user_name}' or Country '{country_name}' not found!")

    # Add relationships between countries and continents they belong to
    def add_belongs_to_continent_relationship(self, country_name, continent_name):
        country = self.matcher.match("Country", name=country_name).first()
        continent = self.matcher.match("Continent", name=continent_name).first()
        if country and continent:
            rel = Relationship(country, "BELONGS_TO", continent)
            self.graph.merge(rel)
            print(f"Country '{country_name}' belongs to continent '{continent_name}'.")
        else:
            print(f"Country '{country_name}' or Continent '{continent_name}' not found!")

    # Add relationships between countries and their official languages
    def add_speaks_language_relationship(self, country_name, language_name):
        country = self.matcher.match("Country", name=country_name).first()
        language = self.matcher.match("Language", name=language_name).first()
        if country and language:
            rel = Relationship(country, "SPEAKS", language)
            self.graph.merge(rel)
            print(f"Country '{country_name}' speaks the language '{language_name}'.")
        else:
            print(f"Country '{country_name}' or Language '{language_name}' not found!")

    # Query: Find countries visited by a specific user
    def find_countries_visited_by_user(self, user_name):
        query = """
        MATCH (u:User)-[:VISITED]->(c:Country)
        WHERE u.name = $user_name
        RETURN c.name AS country
        """
        results = self.graph.run(query, user_name=user_name).data()
        return [record['country'] for record in results]

    # Query: Recommend countries based on continents a user visited
    def recommend_countries_by_continent(self, user_name):
        query = """
        MATCH (u:User)-[:VISITED]->(:Country)-[:BELONGS_TO]->(con:Continent)<-[:BELONGS_TO]-(c:Country)
        WHERE u.name = $user_name
        RETURN DISTINCT c.name AS country
        LIMIT 5
        """
        results = self.graph.run(query, user_name=user_name).data()
        return [record['country'] for record in results]

    # Query: Recommend countries based on languages spoken in countries a user visited
    def recommend_countries_by_language(self, user_name):
        query = """
        MATCH (u:User)-[:VISITED]->(:Country)-[:SPEAKS]->(l:Language)<-[:SPEAKS]-(c:Country)
        WHERE u.name = $user_name
        RETURN DISTINCT c.name AS country
        LIMIT 5
        """
        results = self.graph.run(query, user_name=user_name).data()
        return [record['country'] for record in results]


# Example usage
if __name__ == "__main__":
    kg = KnowledgeGraph()

    # Clear the graph (optional)
    kg.clear_graph()

    # Add Users, Countries, Continents, and Languages
    users = ["Alice", "Bob", "Carol"]
    countries = ["France", "Germany", "Japan", "Brazil"]
    continents = ["Europe", "Asia", "South America"]
    languages = ["French", "German", "Japanese", "Portuguese"]

    kg.add_users(users)
    kg.add_countries(countries)
    kg.add_continents(continents)
    kg.add_languages(languages)

    # Add relationships
    kg.add_visited_relationship("Alice", "France")
    kg.add_visited_relationship("Bob", "Germany")
    kg.add_visited_relationship("Carol", "Japan")

    kg.add_belongs_to_continent_relationship("France", "Europe")
    kg.add_belongs_to_continent_relationship("Germany", "Europe")
    kg.add_belongs_to_continent_relationship("Japan", "Asia")
    kg.add_belongs_to_continent_relationship("Brazil", "South America")

    kg.add_speaks_language_relationship("France", "French")
    kg.add_speaks_language_relationship("Germany", "German")
    kg.add_speaks_language_relationship("Japan", "Japanese")
    kg.add_speaks_language_relationship("Brazil", "Portuguese")

    # Queries
    alice_visited = kg.find_countries_visited_by_user("Alice")
    print("Countries visited by Alice:", alice_visited)

    recommendations_by_continent = kg.recommend_countries_by_continent("Alice")
    print("Countries recommended for Alice (by continent):", recommendations_by_continent)

    recommendations_by_language = kg.recommend_countries_by_language("Alice")
    print("Countries recommended for Alice (by language):", recommendations_by_language)

Graph cleared.
User 'Alice' added.
User 'Bob' added.
User 'Carol' added.


/var/folders/lx/cdwx1p1n5gq00_jxy52yl3wm0000gn/T/ipykernel_2148/1462425018.py:20: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Country 'France' added.
Country 'Germany' added.
Country 'Japan' added.
Country 'Brazil' added.


/var/folders/lx/cdwx1p1n5gq00_jxy52yl3wm0000gn/T/ipykernel_2148/1462425018.py:29: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Continent 'Europe' added.
Continent 'Asia' added.
Continent 'South America' added.


/var/folders/lx/cdwx1p1n5gq00_jxy52yl3wm0000gn/T/ipykernel_2148/1462425018.py:38: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Language 'French' added.
Language 'German' added.
Language 'Japanese' added.
Language 'Portuguese' added.


/var/folders/lx/cdwx1p1n5gq00_jxy52yl3wm0000gn/T/ipykernel_2148/1462425018.py:47: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


User 'Alice' visited the country 'France'.
User 'Bob' visited the country 'Germany'.
User 'Carol' visited the country 'Japan'.
Country 'France' belongs to continent 'Europe'.
Country 'Germany' belongs to continent 'Europe'.
Country 'Japan' belongs to continent 'Asia'.
Country 'Brazil' belongs to continent 'South America'.
Country 'France' speaks the language 'French'.
Country 'Germany' speaks the language 'German'.
Country 'Japan' speaks the language 'Japanese'.
Country 'Brazil' speaks the language 'Portuguese'.
Countries visited by Alice: ['France']
Countries recommended for Alice (by continent): ['Germany']
Countries recommended for Alice (by language): []
